In [1]:
import os.path
import pickle
import re
import json
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from datetime import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
#dfm = pd.read_csv(r"C:\Users\aiah2\Downloads\Python\Project\DATA\mislaka_feb2023.csv")

In [3]:
dfp = pd.read_csv(r"C:\Users\aiah2\Downloads\Python\Project\DATA\Pcount_feb2023.csv")

In [25]:
#dfm.columns

Index(['AgencyName', 'ClusterName', 'AgencyId', 'RouteId', 'RouteShortName',
       'Direction', 'Alternative', 'TripId', 'LicensePlate',
       'PlannedTripStartDate', 'stopCode', 'DayPeriod',
       'Transaction_DayPeriod', 'TripStartDateTime', 'NumOfUsers', 'd_date',
       'i'],
      dtype='object')

In [4]:
dfp.columns

Index(['ValidTrip', 'AgencyId', 'AgencyName', 'ClusterName', 'RouteId',
       'Direction', 'Alternative', 'RouteShortName', 'TripStartDate',
       'TripStartTime', 'PlannedTripStartTime', 'StopCode', 'StopName',
       'StopSequence', 'DayInWeek', 'StationStatus_rb_cd', 'LicensePlate',
       'TripId', 'OpenDoorDateTime', 'CloseDoorDateTime', 'PassengersUpSum',
       'PassengersDownSum', 'PassengersContinue', 'Lat', 'Long'],
      dtype='object')

In [2]:
os.chdir(r"C:\Users\aiah2\Downloads")

In [6]:
#dfm['d_date'] = (pd.to_datetime(dfm['PlannedTripStartDate']).dt.date).astype(str).str.strip()
dfp['d_date'] = (pd.to_datetime(dfp['TripStartDate']).dt.date).astype(str).str.strip()

In [7]:
#dfm['i'] = dfm['TripId'].astype(str) + "_" + dfm['d_date'] + "_" + dfm['stopCode'].astype(str)
dfp['i'] =  dfp['TripId'].astype(str) + "_" + dfp['d_date'] + "_" + dfp['StopCode'].astype(str)

In [8]:
print(dfp.shape)

(1501698, 27)


In [7]:
#result = pd.merge(dfm, dfp, left_on=['i'], right_on = ['i'])

In [7]:
#result.shape

In [9]:
with open('vm_data.pkl', 'rb') as f:
    vm = pickle.load(f)

In [10]:
vm.shape

(4112040, 11)

In [12]:
vm['stopCode'] = vm.Linkref.apply(lambda s: s[:s.find(":")])

In [13]:
vm['nextStopCode'] = vm.Linkref.apply(lambda s: s[s.find(":")+1:])

In [14]:
vm[['stopCode','nextStopCode']] = vm[['stopCode','nextStopCode']].astype(int)

In [15]:
vm['i'] = vm['TripId'].astype(str) + "_" + vm['d_date'] + "_" + vm['stopCode'].astype(str)

In [16]:
vm

,TripId,LINE_SHORT_NAME,OriginAimedDepartureTime,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i
0,0,3,2023-02-01 05:22:00,2,2023-02-01 05:30:00,2023-02-01 05:30:34,3375:1141,0,2023-02-01,NaT,NaT,3375,1141,0_2023-02-01_3375
1,0,34,2023-02-01 05:42:00,2,2023-02-01 05:42:09,2023-02-01 05:42:21,3300:3301,1,2023-02-01,2023-02-01 05:30:34,NaT,3300,3301,0_2023-02-01_3300
2,0,15,2023-02-01 05:45:00,2,2023-02-01 05:45:03,2023-02-01 05:45:20,3300:3301,1,2023-02-01,2023-02-01 05:42:21,2023-02-01 05:30:34,3300,3301,0_2023-02-01_3300
3,0,66,2023-02-01 05:52:00,2,2023-02-01 05:53:48,2023-02-01 05:54:01,1836:5882,1,2023-02-01,2023-02-01 05:45:20,2023-02-01 05:42:21,1836,5882,0_2023-02-01_1836
4,0,39,2023-02-01 06:03:00,2,2023-02-01 06:05:03,2023-02-01 06:05:19,1819:2072,0,2023-02-01,2023-02-01 05:54:01,2023-02-01 05:45:20,1819,2072,0_2023-02-01_1819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4112035,585070645,49,2023-02-26 00:00:00,36,2023-02-26 00:39:19,2023-02-26 00:39:32,1926:1809,1,2023-02-26,2023-02-26 00:38:57,2023-02-26 00:38:37,1926,1809,585070645_2023-02-26_1926
4112036,585070645,49,2023-02-26 00:00:00,37,2023-02-26 00:39:49,2023-02-26 00:40:01,1809:9970,1,2023-02-26,2023-02-26 00:39:32,2023-02-26 00:38:57,1809,9970,585070645_2023-02-26_1809
4112037,585070645,49,2023-02-26 00:00:00,42,2023-02-26 00:41:40,2023-02-26 00:41:58,1488:1812,1,2023-02-26,2023-02-26 00:40:01,2023-02-26 00:39:32,1488,1812,585070645_2023-02-26_1488
4112038,585070645,49,2023-02-26 00:00:00,43,2023-02-26 00:42:34,2023-02-26 00:42:50,1812:60028,1,2023-02-26,2023-02-26 00:41:58,2023-02-26 00:40:01,1812,60028,585070645_2023-02-26_1812


In [17]:
vm.to_pickle("vm_new.pkl")

In [18]:
vm[(vm.TripId == 11855336) & (vm.d_date == '2023-02-28') & (vm.stopCode == 9878)]

,TripId,LINE_SHORT_NAME,OriginAimedDepartureTime,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i
2229948,11855336,29,2023-02-28 15:52:00,29,2023-02-28 16:16:37,2023-02-28 16:16:51,9878:4028,0,2023-02-28,2023-02-28 16:16:23,2023-02-28 16:10:43,9878,4028,11855336_2023-02-28_9878


## Merge Data

In [19]:
data = pd.merge(vm, dfp, left_on=['i'], right_on = ['i'])

In [20]:
data

,TripId_x,LINE_SHORT_NAME,OriginAimedDepartureTime,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date_x,pre_dt,...,LicensePlate,TripId_y,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue,Lat,Long,d_date_y
0,1361133,16,2023-02-19 09:40:00,2,2023-02-19 09:40:54,2023-02-19 09:41:10,1819:2072,1,2023-02-19,NaT,...,7797769,1361133,2023-02-19 09:37:16.000,2023-02-19 09:40:29.000,1,0,1,31.760513,35.183327,2023-02-19
1,1361133,16,2023-02-19 09:40:00,3,2023-02-19 09:41:10,2023-02-19 09:42:15,2072:544,1,2023-02-19,2023-02-19 09:41:10,...,7797769,1361133,2023-02-19 09:41:05.000,2023-02-19 09:41:10.000,1,0,2,31.758865,35.183699,2023-02-19
2,1361133,16,2023-02-19 09:40:00,4,2023-02-19 09:43:06,2023-02-19 09:43:34,544:9893,1,2023-02-19,2023-02-19 09:42:15,...,7797769,1361133,2023-02-19 09:41:59.000,2023-02-19 09:42:15.000,5,0,7,31.758864,35.184640,2023-02-19
3,1361133,16,2023-02-19 09:40:00,5,2023-02-19 09:44:25,2023-02-19 09:44:59,9893:3292,1,2023-02-19,2023-02-19 09:43:34,...,7797769,1361133,2023-02-19 09:43:22.000,2023-02-19 09:43:34.000,3,0,10,31.762116,35.185983,2023-02-19
4,1361133,16,2023-02-19 09:40:00,6,2023-02-19 09:45:42,2023-02-19 09:46:09,3292:557,1,2023-02-19,2023-02-19 09:44:59,...,7797769,1361133,2023-02-19 09:44:45.000,2023-02-19 09:44:59.000,8,0,18,31.763355,35.187256,2023-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141278,585053977,63,2023-02-04 18:25:00,24,2023-02-04 18:44:17,2023-02-04 18:44:39,332:2722,0,2023-02-04,2023-02-04 18:43:54,...,7800569,585053977,NaN,NaN,0,0,4,31.815418,35.199356,2023-02-04
1141279,585053977,63,2023-02-04 18:25:00,25,2023-02-04 18:45:04,2023-02-04 18:45:26,2722:5131,0,2023-02-04,2023-02-04 18:44:39,...,7800569,585053977,2023-02-04 18:44:32.000,2023-02-04 18:44:39.000,0,3,1,31.815387,35.196862,2023-02-04
1141280,585053977,63,2023-02-04 18:25:00,26,2023-02-04 18:46:39,2023-02-04 18:47:04,5131:904,0,2023-02-04,2023-02-04 18:45:26,...,7800569,585053977,2023-02-04 18:45:19.000,2023-02-04 18:45:26.000,0,1,0,31.817319,35.195516,2023-02-04
1141281,585053977,63,2023-02-04 18:25:00,27,2023-02-04 18:47:15,2023-02-04 18:47:37,904:885,0,2023-02-04,2023-02-04 18:47:04,...,7800569,585053977,NaN,NaN,0,0,0,31.821671,35.190186,2023-02-04


In [21]:
data.isnull().sum()

TripId_x                         0
LINE_SHORT_NAME                  0
OriginAimedDepartureTime         0
stopOrder                        0
actualArrivalTime                0
actualDepartureTime              0
Linkref                          0
direction                        0
d_date_x                         0
pre_dt                       38720
pre_dt2                      77297
stopCode                         0
nextStopCode                     0
i                                0
ValidTrip                        0
AgencyId                         0
AgencyName                       0
ClusterName                      0
RouteId                          0
Direction                        0
Alternative                      0
RouteShortName                   0
TripStartDate                    0
TripStartTime                    0
PlannedTripStartTime             0
StopCode                         0
StopName                         0
StopSequence                     0
DayInWeek           

In [24]:
vm.to_pickle("mergedData.pkl")

In [7]:
with open('mergedData.pkl', 'rb') as f:
    data = pickle.load(f)

In [22]:
# without mislaka
data.shape

(1141283, 40)

In [23]:
print(list(data.columns))

['TripId_x', 'LINE_SHORT_NAME', 'OriginAimedDepartureTime', 'stopOrder', 'actualArrivalTime', 'actualDepartureTime', 'Linkref', 'direction', 'd_date_x', 'pre_dt', 'pre_dt2', 'stopCode', 'nextStopCode', 'i', 'ValidTrip', 'AgencyId', 'AgencyName', 'ClusterName', 'RouteId', 'Direction', 'Alternative', 'RouteShortName', 'TripStartDate', 'TripStartTime', 'PlannedTripStartTime', 'StopCode', 'StopName', 'StopSequence', 'DayInWeek', 'StationStatus_rb_cd', 'LicensePlate', 'TripId_y', 'OpenDoorDateTime', 'CloseDoorDateTime', 'PassengersUpSum', 'PassengersDownSum', 'PassengersContinue', 'Lat', 'Long', 'd_date_y']


In [25]:
data.drop(columns = ['ValidTrip', 'AgencyId', 'Alternative', 'LicensePlate', 'AgencyName', 'ClusterName', 'Direction', 
                     'RouteShortName', 'TripStartDate', 'TripStartTime', 'PlannedTripStartTime', 
                     'StopCode', 'StopName', 'StationStatus_rb_cd', 'TripId_y', 
                     'Lat', 'Long', 'd_date_y', 'OriginAimedDepartureTime', 'RouteId'] , axis=1, inplace = True)

In [26]:
data.columns

Index(['TripId_x', 'LINE_SHORT_NAME', 'stopOrder', 'actualArrivalTime',
       'actualDepartureTime', 'Linkref', 'direction', 'd_date_x', 'pre_dt',
       'pre_dt2', 'stopCode', 'nextStopCode', 'i', 'StopSequence', 'DayInWeek',
       'OpenDoorDateTime', 'CloseDoorDateTime', 'PassengersUpSum',
       'PassengersDownSum', 'PassengersContinue'],
      dtype='object')

In [27]:
data.shape

(1141283, 20)

In [28]:
print(list(data.columns))

['TripId_x', 'LINE_SHORT_NAME', 'stopOrder', 'actualArrivalTime', 'actualDepartureTime', 'Linkref', 'direction', 'd_date_x', 'pre_dt', 'pre_dt2', 'stopCode', 'nextStopCode', 'i', 'StopSequence', 'DayInWeek', 'OpenDoorDateTime', 'CloseDoorDateTime', 'PassengersUpSum', 'PassengersDownSum', 'PassengersContinue']


In [29]:
data.rename(columns = {'TripId_x':'tripID', 'd_date_x':'d_date', 'stopCode_x':'stopCode'}, inplace = True)

In [30]:
data

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue
0,1361133,16,2,2023-02-19 09:40:54,2023-02-19 09:41:10,1819:2072,1,2023-02-19,NaT,NaT,1819,2072,1361133_2023-02-19_1819,1,ראשון,2023-02-19 09:37:16.000,2023-02-19 09:40:29.000,1,0,1
1,1361133,16,3,2023-02-19 09:41:10,2023-02-19 09:42:15,2072:544,1,2023-02-19,2023-02-19 09:41:10,NaT,2072,544,1361133_2023-02-19_2072,2,ראשון,2023-02-19 09:41:05.000,2023-02-19 09:41:10.000,1,0,2
2,1361133,16,4,2023-02-19 09:43:06,2023-02-19 09:43:34,544:9893,1,2023-02-19,2023-02-19 09:42:15,2023-02-19 09:41:10,544,9893,1361133_2023-02-19_544,3,ראשון,2023-02-19 09:41:59.000,2023-02-19 09:42:15.000,5,0,7
3,1361133,16,5,2023-02-19 09:44:25,2023-02-19 09:44:59,9893:3292,1,2023-02-19,2023-02-19 09:43:34,2023-02-19 09:42:15,9893,3292,1361133_2023-02-19_9893,4,ראשון,2023-02-19 09:43:22.000,2023-02-19 09:43:34.000,3,0,10
4,1361133,16,6,2023-02-19 09:45:42,2023-02-19 09:46:09,3292:557,1,2023-02-19,2023-02-19 09:44:59,2023-02-19 09:43:34,3292,557,1361133_2023-02-19_3292,5,ראשון,2023-02-19 09:44:45.000,2023-02-19 09:44:59.000,8,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141278,585053977,63,24,2023-02-04 18:44:17,2023-02-04 18:44:39,332:2722,0,2023-02-04,2023-02-04 18:43:54,2023-02-04 18:43:38,332,2722,585053977_2023-02-04_332,23,שבת,NaN,NaN,0,0,4
1141279,585053977,63,25,2023-02-04 18:45:04,2023-02-04 18:45:26,2722:5131,0,2023-02-04,2023-02-04 18:44:39,2023-02-04 18:43:54,2722,5131,585053977_2023-02-04_2722,24,שבת,2023-02-04 18:44:32.000,2023-02-04 18:44:39.000,0,3,1
1141280,585053977,63,26,2023-02-04 18:46:39,2023-02-04 18:47:04,5131:904,0,2023-02-04,2023-02-04 18:45:26,2023-02-04 18:44:39,5131,904,585053977_2023-02-04_5131,25,שבת,2023-02-04 18:45:19.000,2023-02-04 18:45:26.000,0,1,0
1141281,585053977,63,27,2023-02-04 18:47:15,2023-02-04 18:47:37,904:885,0,2023-02-04,2023-02-04 18:47:04,2023-02-04 18:45:26,904,885,585053977_2023-02-04_904,26,שבת,NaN,NaN,0,0,0


In [31]:
len(data.tripID.unique())

19739

In [32]:
data[(data.tripID == 1361360)]

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue
3978,1361360,16,2,2023-02-02 18:41:03,2023-02-02 18:41:18,1819:2072,1,2023-02-02,NaT,NaT,1819,2072,1361360_2023-02-02_1819,1,חמישי,2023-02-02 18:40:24.000,2023-02-02 18:40:46.000,0,0,0
3979,1361360,16,3,2023-02-02 18:41:19,2023-02-02 18:41:35,2072:544,1,2023-02-02,2023-02-02 18:41:18,NaT,2072,544,1361360_2023-02-02_2072,2,חמישי,NaN,NaN,0,0,0
3980,1361360,16,4,2023-02-02 18:42:44,2023-02-02 18:43:10,544:9893,1,2023-02-02,2023-02-02 18:41:35,2023-02-02 18:41:18,544,9893,1361360_2023-02-02_544,3,חמישי,2023-02-02 18:41:22.000,2023-02-02 18:41:46.000,0,0,0
3981,1361360,16,7,2023-02-02 18:45:57,2023-02-02 18:46:47,557:558,1,2023-02-02,2023-02-02 18:43:10,2023-02-02 18:41:35,557,558,1361360_2023-02-02_557,6,חמישי,2023-02-02 18:45:14.000,2023-02-02 18:45:31.000,1,0,17
3982,1361360,16,8,2023-02-02 18:47:21,2023-02-02 18:47:56,558:559,1,2023-02-02,2023-02-02 18:46:47,2023-02-02 18:43:10,558,559,1361360_2023-02-02_558,7,חמישי,2023-02-02 18:46:35.000,2023-02-02 18:46:56.000,8,0,25
3983,1361360,16,9,2023-02-02 18:48:25,2023-02-02 18:48:49,559:199,1,2023-02-02,2023-02-02 18:47:56,2023-02-02 18:46:47,559,199,1361360_2023-02-02_559,8,חמישי,2023-02-02 18:47:42.000,2023-02-02 18:48:13.000,2,4,23
3984,1361360,16,10,2023-02-02 18:49:08,2023-02-02 18:49:34,199:3220,1,2023-02-02,2023-02-02 18:48:49,2023-02-02 18:47:56,199,3220,1361360_2023-02-02_199,9,חמישי,2023-02-02 18:48:42.000,2023-02-02 18:49:00.000,0,1,22
3985,1361360,16,11,2023-02-02 18:50:15,2023-02-02 18:50:31,3220:1222,1,2023-02-02,2023-02-02 18:49:34,2023-02-02 18:48:49,3220,1222,1361360_2023-02-02_3220,10,חמישי,2023-02-02 18:49:28.000,2023-02-02 18:49:54.000,2,0,24
3986,1361360,16,12,2023-02-02 18:52:01,2023-02-02 18:52:28,1222:1589,1,2023-02-02,2023-02-02 18:50:31,2023-02-02 18:49:34,1222,1589,1361360_2023-02-02_1222,11,חמישי,2023-02-02 18:50:24.000,2023-02-02 18:50:43.000,1,0,25
3987,1361360,16,13,2023-02-02 18:54:40,2023-02-02 18:54:59,1589:5980,1,2023-02-02,2023-02-02 18:52:28,2023-02-02 18:50:31,1589,5980,1361360_2023-02-02_1589,12,חמישי,2023-02-02 18:52:14.000,2023-02-02 18:52:49.000,1,15,11


In [33]:
data[(data.tripID == 1361360) & (data.d_date == '2023-02-02')]

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue
3978,1361360,16,2,2023-02-02 18:41:03,2023-02-02 18:41:18,1819:2072,1,2023-02-02,NaT,NaT,1819,2072,1361360_2023-02-02_1819,1,חמישי,2023-02-02 18:40:24.000,2023-02-02 18:40:46.000,0,0,0
3979,1361360,16,3,2023-02-02 18:41:19,2023-02-02 18:41:35,2072:544,1,2023-02-02,2023-02-02 18:41:18,NaT,2072,544,1361360_2023-02-02_2072,2,חמישי,NaN,NaN,0,0,0
3980,1361360,16,4,2023-02-02 18:42:44,2023-02-02 18:43:10,544:9893,1,2023-02-02,2023-02-02 18:41:35,2023-02-02 18:41:18,544,9893,1361360_2023-02-02_544,3,חמישי,2023-02-02 18:41:22.000,2023-02-02 18:41:46.000,0,0,0
3981,1361360,16,7,2023-02-02 18:45:57,2023-02-02 18:46:47,557:558,1,2023-02-02,2023-02-02 18:43:10,2023-02-02 18:41:35,557,558,1361360_2023-02-02_557,6,חמישי,2023-02-02 18:45:14.000,2023-02-02 18:45:31.000,1,0,17
3982,1361360,16,8,2023-02-02 18:47:21,2023-02-02 18:47:56,558:559,1,2023-02-02,2023-02-02 18:46:47,2023-02-02 18:43:10,558,559,1361360_2023-02-02_558,7,חמישי,2023-02-02 18:46:35.000,2023-02-02 18:46:56.000,8,0,25
3983,1361360,16,9,2023-02-02 18:48:25,2023-02-02 18:48:49,559:199,1,2023-02-02,2023-02-02 18:47:56,2023-02-02 18:46:47,559,199,1361360_2023-02-02_559,8,חמישי,2023-02-02 18:47:42.000,2023-02-02 18:48:13.000,2,4,23
3984,1361360,16,10,2023-02-02 18:49:08,2023-02-02 18:49:34,199:3220,1,2023-02-02,2023-02-02 18:48:49,2023-02-02 18:47:56,199,3220,1361360_2023-02-02_199,9,חמישי,2023-02-02 18:48:42.000,2023-02-02 18:49:00.000,0,1,22
3985,1361360,16,11,2023-02-02 18:50:15,2023-02-02 18:50:31,3220:1222,1,2023-02-02,2023-02-02 18:49:34,2023-02-02 18:48:49,3220,1222,1361360_2023-02-02_3220,10,חמישי,2023-02-02 18:49:28.000,2023-02-02 18:49:54.000,2,0,24
3986,1361360,16,12,2023-02-02 18:52:01,2023-02-02 18:52:28,1222:1589,1,2023-02-02,2023-02-02 18:50:31,2023-02-02 18:49:34,1222,1589,1361360_2023-02-02_1222,11,חמישי,2023-02-02 18:50:24.000,2023-02-02 18:50:43.000,1,0,25
3987,1361360,16,13,2023-02-02 18:54:40,2023-02-02 18:54:59,1589:5980,1,2023-02-02,2023-02-02 18:52:28,2023-02-02 18:50:31,1589,5980,1361360_2023-02-02_1589,12,חמישי,2023-02-02 18:52:14.000,2023-02-02 18:52:49.000,1,15,11


In [34]:
data.isnull().sum()

tripID                      0
LINE_SHORT_NAME             0
stopOrder                   0
actualArrivalTime           0
actualDepartureTime         0
Linkref                     0
direction                   0
d_date                      0
pre_dt                  38720
pre_dt2                 77297
stopCode                    0
nextStopCode                0
i                           0
StopSequence                0
DayInWeek                   0
OpenDoorDateTime       401961
CloseDoorDateTime      400481
PassengersUpSum             0
PassengersDownSum           0
PassengersContinue          0
dtype: int64

In [35]:
data.dtypes

tripID                          int64
LINE_SHORT_NAME                object
stopOrder                       int64
actualArrivalTime      datetime64[ns]
actualDepartureTime    datetime64[ns]
Linkref                        object
direction                       int64
d_date                         object
pre_dt                 datetime64[ns]
pre_dt2                datetime64[ns]
stopCode                        int32
nextStopCode                    int32
i                              object
StopSequence                    int64
DayInWeek                      object
OpenDoorDateTime               object
CloseDoorDateTime              object
PassengersUpSum                 int64
PassengersDownSum               int64
PassengersContinue              int64
dtype: object

In [36]:
data = data.dropna()

In [37]:
data.shape

(694759, 20)

In [38]:
data['OpenDoorDateTime'] = data['OpenDoorDateTime'].apply(lambda s: s[:s.find(".000")])
data['CloseDoorDateTime'] = data['CloseDoorDateTime'].apply(lambda s: s[:s.find(".000")])

C:\Users\aiah2\AppData\Local\Temp\ipykernel_9192\1090010311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['OpenDoorDateTime'] = data['OpenDoorDateTime'].apply(lambda s: s[:s.find(".000")])
C:\Users\aiah2\AppData\Local\Temp\ipykernel_9192\1090010311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CloseDoorDateTime'] = data['CloseDoorDateTime'].apply(lambda s: s[:s.find(".000")])


In [39]:
data

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,stopCode,nextStopCode,i,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue
2,1361133,16,4,2023-02-19 09:43:06,2023-02-19 09:43:34,544:9893,1,2023-02-19,2023-02-19 09:42:15,2023-02-19 09:41:10,544,9893,1361133_2023-02-19_544,3,ראשון,2023-02-19 09:41:59,2023-02-19 09:42:15,5,0,7
3,1361133,16,5,2023-02-19 09:44:25,2023-02-19 09:44:59,9893:3292,1,2023-02-19,2023-02-19 09:43:34,2023-02-19 09:42:15,9893,3292,1361133_2023-02-19_9893,4,ראשון,2023-02-19 09:43:22,2023-02-19 09:43:34,3,0,10
4,1361133,16,6,2023-02-19 09:45:42,2023-02-19 09:46:09,3292:557,1,2023-02-19,2023-02-19 09:44:59,2023-02-19 09:43:34,3292,557,1361133_2023-02-19_3292,5,ראשון,2023-02-19 09:44:45,2023-02-19 09:44:59,8,0,18
5,1361133,16,7,2023-02-19 09:46:35,2023-02-19 09:47:26,557:558,1,2023-02-19,2023-02-19 09:46:09,2023-02-19 09:44:59,557,558,1361133_2023-02-19_557,6,ראשון,2023-02-19 09:45:59,2023-02-19 09:46:09,3,0,21
6,1361133,16,8,2023-02-19 09:48:28,2023-02-19 09:49:07,558:559,1,2023-02-19,2023-02-19 09:47:26,2023-02-19 09:46:09,558,559,1361133_2023-02-19_558,7,ראשון,2023-02-19 09:47:05,2023-02-19 09:47:26,6,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141270,585053977,63,16,2023-02-04 18:38:33,2023-02-04 18:38:46,1190:1188,0,2023-02-04,2023-02-04 18:37:42,2023-02-04 18:36:31,1190,1188,585053977_2023-02-04_1190,15,שבת,2023-02-04 18:37:37,2023-02-04 18:37:42,0,1,0
1141275,585053977,63,21,2023-02-04 18:42:17,2023-02-04 18:42:39,2187:5997,0,2023-02-04,2023-02-04 18:41:58,2023-02-04 18:41:22,2187,5997,585053977_2023-02-04_2187,20,שבת,2023-02-04 18:41:51,2023-02-04 18:41:58,3,0,3
1141276,585053977,63,22,2023-02-04 18:43:04,2023-02-04 18:43:38,5997:2190,0,2023-02-04,2023-02-04 18:42:39,2023-02-04 18:41:58,5997,2190,585053977_2023-02-04_5997,21,שבת,2023-02-04 18:42:34,2023-02-04 18:42:39,1,0,4
1141279,585053977,63,25,2023-02-04 18:45:04,2023-02-04 18:45:26,2722:5131,0,2023-02-04,2023-02-04 18:44:39,2023-02-04 18:43:54,2722,5131,585053977_2023-02-04_2722,24,שבת,2023-02-04 18:44:32,2023-02-04 18:44:39,0,3,1


In [240]:
#data['oDoor'] = data['OpenDoorDateTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
#data['cDoor'] = data['CloseDoorDateTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [40]:
data['oDoor'] = data['OpenDoorDateTime'].apply(lambda x: np.datetime64(x).astype('datetime64[ns]'))
data['cDoor'] = data['CloseDoorDateTime'].apply(lambda x: np.datetime64(x).astype('datetime64[ns]'))

C:\Users\aiah2\AppData\Local\Temp\ipykernel_9192\3539325687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['oDoor'] = data['OpenDoorDateTime'].apply(lambda x: np.datetime64(x).astype('datetime64[ns]'))
C:\Users\aiah2\AppData\Local\Temp\ipykernel_9192\3539325687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cDoor'] = data['CloseDoorDateTime'].apply(lambda x: np.datetime64(x).astype('datetime64[ns]'))


In [41]:
data.dtypes

tripID                          int64
LINE_SHORT_NAME                object
stopOrder                       int64
actualArrivalTime      datetime64[ns]
actualDepartureTime    datetime64[ns]
Linkref                        object
direction                       int64
d_date                         object
pre_dt                 datetime64[ns]
pre_dt2                datetime64[ns]
stopCode                        int32
nextStopCode                    int32
i                              object
StopSequence                    int64
DayInWeek                      object
OpenDoorDateTime               object
CloseDoorDateTime              object
PassengersUpSum                 int64
PassengersDownSum               int64
PassengersContinue              int64
oDoor                  datetime64[ns]
cDoor                  datetime64[ns]
dtype: object

In [42]:
data['timeSpent'] = data.apply(lambda x: (x.cDoor - x.oDoor)/np.timedelta64(1, 's'), axis=1)

C:\Users\aiah2\AppData\Local\Temp\ipykernel_9192\2890865404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['timeSpent'] = data.apply(lambda x: (x.cDoor - x.oDoor)/np.timedelta64(1, 's'), axis=1)


In [43]:
data.iloc[597125]

tripID                                 43218305
LINE_SHORT_NAME                              19
stopOrder                                    25
actualArrivalTime           2023-02-08 08:26:38
actualDepartureTime         2023-02-08 08:27:13
Linkref                               2024:3605
direction                                     0
d_date                               2023-02-08
pre_dt                      2023-02-08 08:25:03
pre_dt2                     2023-02-08 08:22:26
stopCode                                   2024
nextStopCode                               3605
i                      43218305_2023-02-08_2024
StopSequence                                 24
DayInWeek                                 רביעי
OpenDoorDateTime            2023-02-08 08:24:52
CloseDoorDateTime           2023-02-08 08:25:03
PassengersUpSum                               1
PassengersDownSum                             2
PassengersContinue                           21
oDoor                       2023-02-08 0

In [44]:
data[(data.tripID == 11855182) & (data.d_date == '2023-02-20') & (data.stopOrder == 7)]

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,...,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue,oDoor,cDoor,timeSpent
597125,11855182,29,7,2023-02-20 08:15:17,2023-02-20 08:15:28,3004:9975,0,2023-02-20,2023-02-20 08:15:15,2023-02-20 08:14:37,...,6,שני,2023-02-20 09:02:52,2023-02-20 08:09:33,1,1,11,2023-02-20 09:02:52,2023-02-20 08:09:33,-3199.0


In [45]:
data.sort_values('timeSpent').head(10)

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,...,StopSequence,DayInWeek,OpenDoorDateTime,CloseDoorDateTime,PassengersUpSum,PassengersDownSum,PassengersContinue,oDoor,cDoor,timeSpent
616367,11855441,29,7,2023-02-08 15:38:38,2023-02-08 15:39:14,3004:9975,0,2023-02-08,2023-02-08 15:38:37,2023-02-08 15:37:56,...,6,רביעי,2023-02-08 16:25:13,2023-02-08 15:31:45,1,0,-1,2023-02-08 16:25:13,2023-02-08 15:31:45,-3208.0
597125,11855182,29,7,2023-02-20 08:15:17,2023-02-20 08:15:28,3004:9975,0,2023-02-20,2023-02-20 08:15:15,2023-02-20 08:14:37,...,6,שני,2023-02-20 09:02:52,2023-02-20 08:09:33,1,1,11,2023-02-20 09:02:52,2023-02-20 08:09:33,-3199.0
615652,11855435,29,7,2023-02-01 14:43:17,2023-02-01 14:43:29,3004:9975,0,2023-02-01,2023-02-01 14:43:16,2023-02-01 14:42:33,...,6,רביעי,2023-02-01 15:28:40,2023-02-01 14:35:24,0,1,-5,2023-02-01 15:28:40,2023-02-01 14:35:24,-3196.0
601009,11855229,29,7,2023-02-13 15:55:31,2023-02-13 15:55:42,3004:9975,0,2023-02-13,2023-02-13 15:55:30,2023-02-13 15:55:00,...,6,שני,2023-02-13 16:39:41,2023-02-13 15:48:07,0,0,6,2023-02-13 16:39:41,2023-02-13 15:48:07,-3094.0
593451,11855122,29,7,2023-02-26 15:54:16,2023-02-26 15:55:14,3004:9975,0,2023-02-26,2023-02-26 15:54:15,2023-02-26 15:53:50,...,6,ראשון,2023-02-26 16:39:51,2023-02-26 15:49:26,0,2,-1,2023-02-26 16:39:51,2023-02-26 15:49:26,-3025.0
601333,11855232,29,7,2023-02-20 16:19:12,2023-02-20 16:19:26,3004:9975,0,2023-02-20,2023-02-20 16:19:11,2023-02-20 16:17:37,...,6,שני,2023-02-20 17:01:07,2023-02-20 16:14:38,1,0,8,2023-02-20 17:01:07,2023-02-20 16:14:38,-2789.0
601094,11855229,29,7,2023-02-27 15:54:51,2023-02-27 15:55:14,3004:9975,0,2023-02-27,2023-02-27 15:54:50,2023-02-27 15:54:16,...,6,שני,2023-02-27 16:32:32,2023-02-27 15:46:09,0,2,2,2023-02-27 16:32:32,2023-02-27 15:46:09,-2783.0
595198,11855145,29,8,2023-02-05 19:32:56,2023-02-05 19:34:33,9975:2711,0,2023-02-05,2023-02-05 19:32:55,2023-02-05 19:32:45,...,7,ראשון,2023-02-05 20:13:02,2023-02-05 19:26:40,1,0,2,2023-02-05 20:13:02,2023-02-05 19:26:40,-2782.0
597915,11855191,29,7,2023-02-20 09:27:03,2023-02-20 09:27:19,3004:9975,0,2023-02-20,2023-02-20 09:27:02,2023-02-20 09:26:12,...,6,שני,2023-02-20 10:05:52,2023-02-20 09:20:54,0,4,-1,2023-02-20 10:05:52,2023-02-20 09:20:54,-2698.0
593410,11855122,29,7,2023-02-19 15:55:22,2023-02-19 15:55:35,3004:9975,0,2023-02-19,2023-02-19 15:55:21,2023-02-19 15:54:28,...,6,ראשון,2023-02-19 16:34:58,2023-02-19 15:50:08,2,2,3,2023-02-19 16:34:58,2023-02-19 15:50:08,-2690.0


In [46]:
data.to_pickle("finalMergedData.pkl")

In [3]:
with open('finalMergedData.pkl', 'rb') as f:
    data = pickle.load(f)

### Encoding Categorial Variables:
1. Linkref
2. DayInWeek
3. DayTime

In [47]:
#data = pd.get_dummies(data, columns = ['Linkref'])

In [4]:
my_dict={'ראשון':'sunday',
         'שני':'monday',
        'שלישי':'tuesday',
        'רביעי':'wednesday',
        'חמישי':'thursday',
        'שישי':'friday',
        'שבת':'saturday',}

data.DayInWeek = data.DayInWeek.replace(my_dict)

In [5]:
data.DayInWeek.unique()

array(['sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday',
       'saturday'], dtype=object)

In [6]:
data = pd.get_dummies(data, columns = ['DayInWeek'])

In [7]:
data.head(10)

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,...,oDoor,cDoor,timeSpent,DayInWeek_friday,DayInWeek_monday,DayInWeek_saturday,DayInWeek_sunday,DayInWeek_thursday,DayInWeek_tuesday,DayInWeek_wednesday
2,1361133,16,4,2023-02-19 09:43:06,2023-02-19 09:43:34,544:9893,1,2023-02-19,2023-02-19 09:42:15,2023-02-19 09:41:10,...,2023-02-19 09:41:59,2023-02-19 09:42:15,16.0,0,0,0,1,0,0,0
3,1361133,16,5,2023-02-19 09:44:25,2023-02-19 09:44:59,9893:3292,1,2023-02-19,2023-02-19 09:43:34,2023-02-19 09:42:15,...,2023-02-19 09:43:22,2023-02-19 09:43:34,12.0,0,0,0,1,0,0,0
4,1361133,16,6,2023-02-19 09:45:42,2023-02-19 09:46:09,3292:557,1,2023-02-19,2023-02-19 09:44:59,2023-02-19 09:43:34,...,2023-02-19 09:44:45,2023-02-19 09:44:59,14.0,0,0,0,1,0,0,0
5,1361133,16,7,2023-02-19 09:46:35,2023-02-19 09:47:26,557:558,1,2023-02-19,2023-02-19 09:46:09,2023-02-19 09:44:59,...,2023-02-19 09:45:59,2023-02-19 09:46:09,10.0,0,0,0,1,0,0,0
6,1361133,16,8,2023-02-19 09:48:28,2023-02-19 09:49:07,558:559,1,2023-02-19,2023-02-19 09:47:26,2023-02-19 09:46:09,...,2023-02-19 09:47:05,2023-02-19 09:47:26,21.0,0,0,0,1,0,0,0
7,1361133,16,9,2023-02-19 09:49:33,2023-02-19 09:50:17,559:199,1,2023-02-19,2023-02-19 09:49:07,2023-02-19 09:47:26,...,2023-02-19 09:48:51,2023-02-19 09:49:07,16.0,0,0,0,1,0,0,0
8,1361133,16,10,2023-02-19 09:50:33,2023-02-19 09:51:04,199:3220,1,2023-02-19,2023-02-19 09:50:17,2023-02-19 09:49:07,...,2023-02-19 09:49:55,2023-02-19 09:50:17,22.0,0,0,0,1,0,0,0
9,1361133,16,11,2023-02-19 09:52:20,2023-02-19 09:52:40,3220:1222,1,2023-02-19,2023-02-19 09:51:04,2023-02-19 09:50:17,...,2023-02-19 09:50:46,2023-02-19 09:51:24,38.0,0,0,0,1,0,0,0
10,1361133,16,12,2023-02-19 09:53:17,2023-02-19 09:53:44,1222:1589,1,2023-02-19,2023-02-19 09:52:40,2023-02-19 09:51:04,...,2023-02-19 09:52:30,2023-02-19 09:52:40,10.0,0,0,0,1,0,0,0
11,1361133,16,13,2023-02-19 09:55:01,2023-02-19 09:55:26,1589:5980,1,2023-02-19,2023-02-19 09:53:44,2023-02-19 09:52:40,...,2023-02-19 09:53:30,2023-02-19 09:53:44,14.0,0,0,0,1,0,0,0


In [8]:
data.columns

Index(['tripID', 'LINE_SHORT_NAME', 'stopOrder', 'actualArrivalTime',
       'actualDepartureTime', 'Linkref', 'direction', 'd_date', 'pre_dt',
       'pre_dt2', 'stopCode', 'nextStopCode', 'i', 'StopSequence',
       'OpenDoorDateTime', 'CloseDoorDateTime', 'PassengersUpSum',
       'PassengersDownSum', 'PassengersContinue', 'oDoor', 'cDoor',
       'timeSpent', 'DayInWeek_friday', 'DayInWeek_monday',
       'DayInWeek_saturday', 'DayInWeek_sunday', 'DayInWeek_thursday',
       'DayInWeek_tuesday', 'DayInWeek_wednesday'],
      dtype='object')

# --------------------------------------------------------------------------------

In [9]:
# Calculate the number of seconds since the start of the day
data['targetTime'] = data['actualArrivalTime'].dt.hour * 3600 + data['actualArrivalTime'].dt.minute * 60 + data['actualArrivalTime'].dt.second

In [10]:
# Calculate the number of seconds since the start of the day
data['preD1'] = data['pre_dt'].dt.hour * 3600 + data['pre_dt'].dt.minute * 60 + data['pre_dt'].dt.second
data['preD2'] = data['pre_dt2'].dt.hour * 3600 + data['pre_dt2'].dt.minute * 60 + data['pre_dt2'].dt.second

In [11]:
data

,tripID,LINE_SHORT_NAME,stopOrder,actualArrivalTime,actualDepartureTime,Linkref,direction,d_date,pre_dt,pre_dt2,...,DayInWeek_friday,DayInWeek_monday,DayInWeek_saturday,DayInWeek_sunday,DayInWeek_thursday,DayInWeek_tuesday,DayInWeek_wednesday,targetTime,preD1,preD2
2,1361133,16,4,2023-02-19 09:43:06,2023-02-19 09:43:34,544:9893,1,2023-02-19,2023-02-19 09:42:15,2023-02-19 09:41:10,...,0,0,0,1,0,0,0,34986,34935,34870
3,1361133,16,5,2023-02-19 09:44:25,2023-02-19 09:44:59,9893:3292,1,2023-02-19,2023-02-19 09:43:34,2023-02-19 09:42:15,...,0,0,0,1,0,0,0,35065,35014,34935
4,1361133,16,6,2023-02-19 09:45:42,2023-02-19 09:46:09,3292:557,1,2023-02-19,2023-02-19 09:44:59,2023-02-19 09:43:34,...,0,0,0,1,0,0,0,35142,35099,35014
5,1361133,16,7,2023-02-19 09:46:35,2023-02-19 09:47:26,557:558,1,2023-02-19,2023-02-19 09:46:09,2023-02-19 09:44:59,...,0,0,0,1,0,0,0,35195,35169,35099
6,1361133,16,8,2023-02-19 09:48:28,2023-02-19 09:49:07,558:559,1,2023-02-19,2023-02-19 09:47:26,2023-02-19 09:46:09,...,0,0,0,1,0,0,0,35308,35246,35169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141270,585053977,63,16,2023-02-04 18:38:33,2023-02-04 18:38:46,1190:1188,0,2023-02-04,2023-02-04 18:37:42,2023-02-04 18:36:31,...,0,0,1,0,0,0,0,67113,67062,66991
1141275,585053977,63,21,2023-02-04 18:42:17,2023-02-04 18:42:39,2187:5997,0,2023-02-04,2023-02-04 18:41:58,2023-02-04 18:41:22,...,0,0,1,0,0,0,0,67337,67318,67282
1141276,585053977,63,22,2023-02-04 18:43:04,2023-02-04 18:43:38,5997:2190,0,2023-02-04,2023-02-04 18:42:39,2023-02-04 18:41:58,...,0,0,1,0,0,0,0,67384,67359,67318
1141279,585053977,63,25,2023-02-04 18:45:04,2023-02-04 18:45:26,2722:5131,0,2023-02-04,2023-02-04 18:44:39,2023-02-04 18:43:54,...,0,0,1,0,0,0,0,67504,67479,67434


In [12]:
data.to_pickle("finalData.pkl")